In [ ]:
!pip install stable-baselines3[extra]
!pip install pettingzoo
!pip install sb3_contrib
!pip install supersuit
!pip install pymunk
!pip install pygame

import stable_baselines3 as SB3
print(SB3.__version__)
import pettingzoo as PET
print(PET.__version__)
import sb3_contrib
print(sb3_contrib.__version__)

In [ ]:
import pettingzoo
from pettingzoo import ParallelEnv
from pettingzoo import AECEnv
from pettingzoo.utils import parallel_to_aec, wrappers, agent_selector
from pettingzoo.utils import ClipOutOfBoundsWrapper
from pettingzoo.utils.conversions import parallel_wrapper_fn
from pettingzoo.sisl._utils import Agent
import gymnasium
from gymnasium.utils import EzPickle ,seeding
from gymnasium.spaces import Discrete, MultiDiscrete, Dict, Box
from gymnasium import spaces
import random
import functools
import numpy as np
import pandas as pd
from copy import copy
from math import pi, sin
from random import gauss, uniform
from tabulate import tabulate
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Required imports
import numpy as np
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector
from gym import spaces
#----------------------------------------------------------------------------------------------------------
# Custom environment class extending PettingZoo's AECEnv
class TehranWaterManagementEnv(AECEnv):
    metadata = {'render.modes': ['human']}

#----------------------------------------------------------------------------------------------------------
    def __init__(self):
        super().__init__()

        # Initialize environment state
        # priority handling
        self.agents = ["industry_1", "industry_2", "industry_3", "municipal_1", "municipal_2", "municipal_3", "municipal_4", "municipal_5", "farmer_1", "farmer_2"]
        self.agent_name_mapping = dict(zip(self.agents, list(range(len(self.agents)))))
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

        # Groundwater Aquifer Initialization
        self.groundwater_levels = np.array([1174, 1058, 1001])  # Initial water levels for areas 1, 2, 3
        self.nitrate_density = np.array([1, 1, 1])  # Initial nitrate density for each area
        self.area_thresholds = np.array([89, 86, 38])  # Thresholds for each area
        self.incoming_water_flow = self._random_incoming_water_flow()  # Initialize random incoming water flow
        self.withdrawal_limits = {"farmer_1": 50e6, "farmer_2": 50e6,
                                  "industry_1": 0.19 * 20.09e6, "industry_2": 0.69 * 20.09e6, "industry_3": 0.12 * 20.09e6,
                                  "municipal_1": 40, "municipal_2": 40, "municipal_3": 40, "municipal_4": 40, "municipal_5": 40}  # Example limits

        # Define observation and action spaces for all agents
        self.action_spaces = {agent: self._define_action_space(agent) for agent in self.agents}
        self.observation_spaces = {agent: self._define_observation_space(agent) for agent in self.agents}

        # Environment Parameters & States
        self.current_step = 0
        self.max_steps = 25
        self.terminate = False
        self.rewards = {agent: 0 for agent in self.agents}
        self.illegal_withdrawal_fines = {agent: 0 for agent in self.agents}
        self.actual_water_supply_percentage = {agent: 100 for agent in self.agents}  # Assuming 100% needs met initially
        self.water_demand = {agent: 0 for agent in self.agents}

        # Crop Information & Economic Parameters
        self.crop_info = {"wheat": {"PPC": 12705, "CPD": 0.63, "Eff": 5352},
                          "barley": {"PPC": 10028, "CPD": 0.70, "Eff": 3840},
                          "alfalfa": {"PPC": 7700, "CPD": 1.09, "Eff": 12000}}

        self.P_OT = 75000  # Price of water per cubic meter (Rials)
        self.PE_1 = 30000  # Price of electricity per kWh (Rials)

        # Municipal Green Space Parameters
        self.potential_green_space = {"municipal_1": 200000, "municipal_2": 150000, "municipal_3": 180000, "municipal_4": 160000, "municipal_5": 140000}
        self.current_green_space = {"municipal_1": 100000, "municipal_2": 75000, "municipal_3": 90000, "municipal_4": 80000, "municipal_5": 70000}

        # Water Allocation & Legal Withdrawal Limits
        self.water_allocation = self._init_water_allocation()
#----------------------------------------------------------------------------------------------------------
    def _random_incoming_water_flow(self):
        # Generates random incoming water flow for each area of the aquifer
        return np.random.uniform(0.1, 10, size=3)
#----------------------------------------------------------------------------------------------------------
    def _define_action_space(self, agent):
        if "farmer" in agent:
            # Example: 3 actions for crop selection, 1 for water needed, 1 for illegal withdrawal
            # Assuming discrete actions for simplicity
            return spaces.MultiDiscrete([3, 100, 1])  # 3 crops, 0-100% water, 0 or 1 for illegal withdrawal
        elif "industry" in agent:
            # Example: 1 action for illegal withdrawal
            return spaces.Discrete(2)  # 0 or 1 for illegal withdrawal
        elif "municipal" in agent:
            # Example: 1 for water needed, 1 for illegal withdrawal, 1 for green space expansion
            return spaces.MultiDiscrete([100, 1, 100])  # 0-100% water needed, 0 or 1 illegal, 0-100% green space expansion
        else:
            raise NotImplementedError(f"Unknown agent type: {agent}")
#----------------------------------------------------------------------------------------------------------
    def _define_observation_space(self, agent):
        groundwater_level_bins = 500  # Example: 500 discrete levels
        # Additional observations can be added as needed, adjust according to your scenario.
        if "farmer" in agent or "industry" in agent or "municipal" in agent:
            # Assuming the only observation for simplicity is the groundwater level.
            # If there are other observations, consider using MultiDiscrete or a similar approach.
            return spaces.Discrete(groundwater_level_bins)
        else:
            raise NotImplementedError(f"Unknown agent type: {agent}")
#----------------------------------------------------------------------------------------------------------
    def _init_water_allocation(self):
        water_allocation = {
            "farmer_1": {"allocated_water": 100, "legal_withdrawal_limit": 50},  # Example values
            "farmer_2": {"allocated_water": 80, "legal_withdrawal_limit": 50},
            "industry_1": {"allocated_water": 380, "legal_withdrawal_limit": 0.4 * 380},  # 19% of total, 40% withdrawal
            "industry_2": {"allocated_water": 1382, "legal_withdrawal_limit": 0.4 * 1382},  # 69% of total, 40% withdrawal
            "industry_3": {"allocated_water": 238, "legal_withdrawal_limit": 0.4 * 238},  # 12% of total, 40% withdrawal
            "municipal_1": {"allocated_water": 200, "legal_withdrawal_limit": 0.4 * 200},  # Example values
            "municipal_2": {"allocated_water": 200, "legal_withdrawal_limit": 0.4 * 200},
            "municipal_3": {"allocated_water": 200, "legal_withdrawal_limit": 0.4 * 200},
            "municipal_4": {"allocated_water": 200, "legal_withdrawal_limit": 0.4 * 200},
            "municipal_5": {"allocated_water": 200, "legal_withdrawal_limit": 0.4 * 200}
        }
        return water_allocation
#----------------------------------------------------------------------------------------------------------
    def update_aquifer_state(self, actions):
        # Constants and area parameters
        area = np.array([78_125_000, 242_750_000, 133_187_500])  # Area of regions 1, 2, and 3
        specific_yield = np.array([0.15, 0.15, 0.15])  # Assuming a constant specific yield for simplicity
        length = np.array([7500, 16000, 12000])  # Length of each region
        hydraulic_conductivity = np.array([0.002, 0.002, 0.002])  # Assuming a constant value for simplicity
        b = np.array([14250, 15750])  # Width between regions
        S = specific_yield  # Specific yield, using the same value for simplicity

        # Update incoming water flow for the new timestep
        self.incoming_water_flow = self._random_incoming_water_flow()

        # Calculate Di_T, Re_T, and Tr for each area
        Di_T = np.zeros(3)  # Total water extracted from each region
        Re_T = np.zeros(3)  # Total recharge in each region
        Tr = np.zeros(2)    # Water transfer between regions


        Re_T = np.array([1e5, 2e5, 1.5e5])  # Just for example purposes
        Di_T = np.array([1e4, 1e4, 1e4])    # Example withdrawal, should be derived from `actions`

        # Calculating water transfer (Tr) based on the formula provided
        for i in range(2):
            Tr[i] = 365 * ((self.groundwater_levels[i] - self.groundwater_levels[i + 1]) / ((length[i] + length[i + 1]) / 2)) * (length[i] + length[i + 1]) / ((length[i] / hydraulic_conductivity[i]) + (length[i + 1] / hydraulic_conductivity[i + 1])) * b[i] * ((self.groundwater_levels[i] + self.groundwater_levels[i + 1]) / 2)

        # Update groundwater levels (B_(n,i+1)) for each area based on water balance equation
        for n in range(3):
            An = area[n]
            B_n_i = self.groundwater_levels[n]  # Current groundwater level for area n
            Tr_in = Tr[n - 1] if n > 0 else 0  # Incoming transfer (none for the first area)
            Tr_out = Tr[n] if n < 2 else 0    # Outgoing transfer (none for the last area)
            Recharge = Re_T[n]  # Total recharge for area n
            Withdrawal = Di_T[n]  # Total withdrawal from area n

            # Water balance equation to calculate the new groundwater level
            B_n_i_plus_1 = B_n_i + (Recharge - Withdrawal + Tr_in - Tr_out) / (S[n] * An)
            self.groundwater_levels[n] = max(B_n_i_plus_1, self.area_thresholds[n])  # Ensuring not falling below threshold

        # Threshold check and adjustments
        for n, level in enumerate(self.groundwater_levels):
            if level < [89, 86, 38][n]:  # Threshold levels for each area
                print(f"Warning: Water level in Area {n+1} below threshold!")
        print("Updated Groundwater levels: ",self.groundwater_levels)
#----------------------------------------------------------------------------------------------------------
    def step(self, action):
        # Store the last action for rendering purposes
        self.last_action = action

        # Check the agent type and handle the action accordingly
        if self.agent_selection.startswith("farmer"):
            self._handle_farmer_action(action)
            # Assuming action structure for farmers is (crop_allocation, requested_water, illegal_withdrawal)
        elif self.agent_selection.startswith("industry"):
            self._handle_industry_action(action)
            # Industry actions do not have a second element for water requested, handle accordingly
        elif self.agent_selection.startswith("municipal"):
            self._handle_municipal_action(action)

        # Update aquifer state after handling the action
        self.update_aquifer_state(action)

        # Update rewards and prepare for the next step
        self.agent_selection = self._agent_selector.next()
        self.current_step += 1  # Move to the next step

        # Check for termination condition
        self._check_termination_conditions()
#----------------------------------------------------------------------------------------------------------
    def _handle_farmer_action(self, action):
        # Extract action components for a farmer
        crop_percentages, requested_water, illegal_withdrawal = action

        # Identifying the farmer's area based on the agent_selection
        area_idx = 1 if self.agent_selection == "farmer_1" else 2  # Adjusting based on your setup

        # Calculate the water demand (DF) for the farmer's crops
        area_hectares = 5103.22 if area_idx == 1 else 3127.78  # Area in hectares for farmer 1 and 2
        water_demand = 0  # Initialize water demand

        for i, crop in enumerate(["wheat", "barley", "alfalfa"]):
            crop_info = self.crop_info[crop]
            crop_demand = (crop_percentages[i] / 100.0) * area_hectares * crop_info['Eff'] / crop_info['CPD']
            water_demand += crop_demand

        # Calculate the actual water available (AF) for the farmer
        allocated_water = self.water_allocation[self.agent_selection]["allocated_water"]
        legal_withdrawal_limit = self.water_allocation[self.agent_selection]["legal_withdrawal_limit"]
        actual_water_available = allocated_water + (legal_withdrawal_limit if not illegal_withdrawal else requested_water)
        actual_water_available = min(actual_water_available, self.groundwater_levels[area_idx])  # Ensure it doesn't exceed the groundwater level

        # Update groundwater level based on the actual water used
        self.groundwater_levels[area_idx] -= actual_water_available

        # Calculate the reward based on the farmer's action and update water demand
        if water_demand > actual_water_available and illegal_withdrawal:
            self.rewards[self.agent_selection] = -1
        else:
            # Simplify profit calculation for demonstration purposes
            profit = actual_water_available * sum(
                (crop_percentages[i] / 100.0) * self.crop_info[crop]['PPC'] * self.crop_info[crop]['CPD']
                for i, crop in enumerate(["wheat", "barley", "alfalfa"])
            )
            # Calculate reward based on profit, adjusting for water price and energy cost
            self.rewards[self.agent_selection] = profit / (self.P_OT + self.PE_1 * self.groundwater_levels[area_idx])
#----------------------------------------------------------------------------------------------------------
    def _handle_industry_action(self, action):
        # Illegal withdrawal is a boolean: 0 or 1
        illegal_withdrawal = action

        # Industry water needs (percentage of total industrial water need)
        industry_water_needs = {
            "industry_1": 0.19 * 20.09,  # 19%
            "industry_2": 0.69 * 20.09,  # 69%
            "industry_3": 0.12 * 20.09,  # 12%
        }

        # Calculating the demand (DW) and allocated water (AW) for the industry
        DW = industry_water_needs[self.agent_selection]  # Demand in million cubic meters
        AW = self.water_allocation[self.agent_selection]["allocated_water"]  # Allocated water from the initial setup

        # Actual water that can be legally withdrawn, based on the agent's allocation and the environment
        legal_withdrawal_limit = self.water_allocation[self.agent_selection]["legal_withdrawal_limit"]
        actual_withdrawal_capacity = AW + legal_withdrawal_limit

        # Determine if the demand exceeds what is legally available, including potential illegal withdrawal
        exceeds_demand = DW > actual_withdrawal_capacity

        # If illegal withdrawal is made and demand exceeds the legal limit
        if illegal_withdrawal and exceeds_demand:
            self.rewards[self.agent_selection] = -1  # Penalize for illegal action
        else:
            # Calculate the reward based on the difference between demand and the water supplied
            # Ensure that actual_withdrawal does not exceed the DW even when illegal withdrawal is considered
            actual_withdrawal = min(DW, actual_withdrawal_capacity) if not illegal_withdrawal else min(DW, AW + 0.4 * DW)
            self.rewards[self.agent_selection] = 1 - abs(DW - actual_withdrawal) / DW

        # Update the groundwater level based on actual withdrawal
        # Assuming an industry mapping to area indices: Industry 1 -> Area 1, etc.
        area_idx = int(self.agent_selection.split("_")[1]) - 1
        self.groundwater_levels[area_idx] -= actual_withdrawal
#----------------------------------------------------------------------------------------------------------
    def _handle_municipal_action(self, action):
        # Unpack the action components for clarity
        water_needed, illegal_withdrawal, green_space_expansion_percentage = action
        agent_region = self.agent_selection

        # Assuming regions 5, 22 are in Area 1, regions 9, 18 in Area 2, and region 21 partially in Areas 1 and 2
        area_index_mapping = {"municipal_1": 0, "municipal_2": 1, "municipal_3": 1, "municipal_4": 0, "municipal_5": 0}
        area_index = area_index_mapping[agent_region]

        # Calculate the maximum possible green space expansion for this step
        Delta_G_max = (self.potential_green_space[agent_region] - self.current_green_space[agent_region]) / 25
        Area_G_current = self.current_green_space[agent_region] + green_space_expansion_percentage * Delta_G_max

        # Calculate the total water demand for the green space
        DG = Area_G_current * 1.5  # Assuming 1.5 cubic meters of water per square meter of green space

        # Calculate the actual water available for the municipal agent
        allocated_water = self.water_allocation[agent_region]["allocated_water"]
        legal_withdrawal_limit = self.water_allocation[agent_region]["legal_withdrawal_limit"]
        actual_water_available = min(allocated_water + legal_withdrawal_limit, self.groundwater_levels[area_index])

        # Determine the reward based on the water demand and the actual water available
        if DG > actual_water_available and illegal_withdrawal:
            self.rewards[agent_region] = -1
        else:
            AG = min(DG, actual_water_available)  # The actual water used is the lesser of the demand or the available
            reward_factor = (1 - abs(DG - AG) / DG)
            green_space_factor = (Area_G_current - self.current_green_space[agent_region]) / (self.potential_green_space[agent_region] - self.current_green_space[agent_region])
            self.rewards[agent_region] = reward_factor * green_space_factor * 100

        # Update the groundwater level based on actual water used
        self.groundwater_levels[area_index] -= AG if illegal_withdrawal else 0
        self.current_green_space[agent_region] = Area_G_current
#----------------------------------------------------------------------------------------------------------
    def _check_termination_conditions(self):
        # Check for groundwater levels below minimum thresholds
        bedrock_levels = np.array([1077, 962, 959])
        min_levels_above_bedrock = np.array([89, 86, 38])
        water_levels_above_bedrock = self.groundwater_levels - bedrock_levels

        if any(water_levels_above_bedrock < min_levels_above_bedrock):
            self.terminate = True
            # Apply penalty to all agents for groundwater level falling below threshold
            for agent in self.agents:
                self.rewards[agent] -= 1

        # Check if max steps have been reached
        if self.current_step >= self.max_steps:
            self.terminate = True


        # Placeholder: Terminate after one round of actions for testing
        # In a real scenario, this should check for the end of a 25-year period or other termination conditions
        self.terminated = True
#----------------------------------------------------------------------------------------------------------
    def reset(self):
        # Reinitialize environment state to start a new episode

        # Reset groundwater levels to initial values
        self.groundwater_levels = np.array([1174, 1058, 1001])

        # Reset water allocation based on initial settings
        self.water_allocation = self._init_water_allocation()

        # Reset rewards for all agents
        self.rewards = {agent: 0 for agent in self.agents}

        # Reset the nitrate density to initial values
        self.nitrate_density = [1, 1, 1]

        # Reset actual water supply percentages to 100%
        self.actual_water_supply_percentage = {agent: 100 for agent in self.agents}

        # Reset tracking of water demand for each agent
        self.water_demand = {agent: 0 for agent in self.agents}

        # Reset current step and termination flag
        self.current_step = 0
        self.terminate = False

        # Reset current green space area to initial values
        self.current_green_space = {
            "municipal_1": 100000,
            "municipal_2": 75000,
            "municipal_3": 90000,
            "municipal_4": 80000,
            "municipal_5": 70000,
        }

        # Reset agent selection for the start of the episode
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

        # This aligns with practices in some RL environments where the reset method also provides the initial state observation
        return self.observe(self.agent_selection)
#----------------------------------------------------------------------------------------------------------
    def render(self, mode='human'):
        if mode == 'human':

            print()
            print("Current Environment step:", self.current_step)
            # Groundwater levels for each area
            for i, level in enumerate(self.groundwater_levels):
                print(f"Groundwater Level Area {i+1}: {level} million cubic meters")

            print()
            # Nitrate density for each area
            for i, density in enumerate(self.nitrate_density):
                print(f"Nitrate Density Area {i+1}: {density} mg/L")

            print()
            # Water allocation and demand for each agent
            print("Water Allocation and Usage:")
            for agent, allocation in self.water_allocation.items():
                print(f"  {agent}: Allocated {allocation['allocated_water']} million cubic meters, Legal Withdrawal Limit {allocation['legal_withdrawal_limit']} million cubic meters")

            print()
            # Current green space for municipal agents
            print("Green Spaces Managed by Municipal Agents:")
            for agent, area in self.current_green_space.items():
                if agent.startswith("municipal"):
                    print(f"  {agent}: Current Green Space Area: {area} square meters")

            print()
            print("Rewards:")
            for agent, reward in self.rewards.items():
                print(f"  {agent}: {reward}")
#----------------------------------------------------------------------------------------------------------
    def observe(self, agent):
        # Groundwater levels per area
        groundwater_observation = np.array(self.groundwater_levels)

        return groundwater_observation
#----------------------------------------------------------------------------------------------------------
    def close(self):
        # Optionally implement any necessary cleanup
        pass
#----------------------------------------------------------------------------------------------------------
# Initialize the environment
env = TehranWaterManagementEnv()

env.reset()
# Run a test step for each agent
# Revised test loop with appropriate default actions for each agent type
for agent in env.agents:
    print("=======================================================")
    print(f"Testing {agent}")
    observation_before = env.observe(agent)
    print(f"Initial observation for {agent}: {observation_before}")

    # Provide default actions based on agent type
    if "farmer" in agent:
        action = ([50, 30, 20], 25, 0)  # Assuming 50% wheat, 30% barley, 20% alfalfa, 25 million cubic meters requested, no illegal withdrawal
    elif "industry" in agent:
        action = (1,)  # Assuming an example action for industry agents
    elif "municipal" in agent:
        action = (15, 0, 5)  # Assuming an example action for municipal agents

    print("Action: ", action)
    env.step(action)
    observation_after = env.observe(agent)
    print(f"Observation after action for {agent}: {observation_after}")
    env.render()
    print("=======================================================")
#----------------------------------------------------------------------------------------------------------

Testing industry_1
Initial observation for industry_1: [1174 1058 1001]
Action:  (1,)
Updated Groundwater levels:  [1169 1058 1001]
Observation after action for industry_1: [1169 1058 1001]

Current Environment step: 1
Groundwater Level Area 1: 1169 million cubic meters
Groundwater Level Area 2: 1058 million cubic meters
Groundwater Level Area 3: 1001 million cubic meters

Nitrate Density Area 1: 1 mg/L
Nitrate Density Area 2: 1 mg/L
Nitrate Density Area 3: 1 mg/L

Water Allocation and Usage:
  farmer_1: Allocated 100 million cubic meters, Legal Withdrawal Limit 50 million cubic meters
  farmer_2: Allocated 80 million cubic meters, Legal Withdrawal Limit 50 million cubic meters
  industry_1: Allocated 380 million cubic meters, Legal Withdrawal Limit 152.0 million cubic meters
  industry_2: Allocated 1382 million cubic meters, Legal Withdrawal Limit 552.8000000000001 million cubic meters
  industry_3: Allocated 238 million cubic meters, Legal Withdrawal Limit 95.2 million cubic meters
 

---